## Face Area Calcuator

In the following example we're going to use a Boon Function to calculate the area, in pixels, of the largest face detected in an image, then we will save that value into a custom field.   This allows us to filter our library of assets to show the best faces to use for recognition training.

A Boon Function is comprised of Python code that gets deployed as a custom model.  The function is called for each Asset that is imported into the system.


In [ ]:
# Take a look at our Boon Fuction
!cat function.py

## Packaging a Boon Function

In order to deploy our Boon Function we simply have to package it in zip file.  This is pretty much true for all uploadable models.

!zip model.zip function.py

## Creating the Custom Field

As you can see from our Boon Fuction code we're setting the value of the 'face_bbox_area' field to the area in pixels of the largest face.  However before that will actually work we need to create the custom field in our project.  The area in pixels is an integer so the type of our field will be integer as well.

In [8]:
import boonsdk
app = boonsdk.app_from_keyfile("apikey.json")

In [ ]:
app.fields.create_field("face_bbox_area", "integer")

In [ ]:
## Creating the Model

Before we can deploy our fuction we must create a Model in BoonAI with the proper type.  In this case we require our fuction to
run AFTER the boonai-face-detection, so setting it as a dependency means that boonai-face-detection will automaticlaly be included before our
face-area-calc module in the asset processing pipeline.

In [9]:
try:
    model = app.models.create_model(
        "face-area-calc",
        boonsdk.ModelType.BOON_FUNCTION,
        dependencies=["boonai-face-detection"]
    )
except boonsdk.BoonSdkDuplicateException:
    model = app.models.get_model("face-area-calc")

## Deploying

To deploy, we just call the app.models.upload_pretrained_model() function with our instance of a boonsdk.Model and the
model.zip file we just made.  The state of the model will change to 'Deploying' and once it is deployed will change to 'Deployed'.

Note that it may take up to 2 minutes before the model is deployed and ready for use.  When you redeploy a new version of
your function, the old version will be served until your new version is fully deployed.


In [ ]:
app.models.upload_pretrained_model(model, "model.zip")

In [ ]:
def logging_callback(model):
     print(model.state)

app.models.wait_on_deploy(model, callback=logging_callback)


In [ ]:
rsp = app.assets.batch_upload_files(
    boonsdk.FileUpload("face.jpg"), modules=["face-area-calc"])

if app.jobs.wait_on_job(rsp):
    asset_id = rsp['created'][0]
    asset = app.assets.get_asset(asset_id)
    #
    # Print the area of the face
    # 
    print(asset.get_attr("custom.face_bbox_area"))
else:
    print("Oh no our job failed!")
    